インポート

In [1]:
# 必要なライブラリのインポート
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pprint import pprint

スクレイピングするURL

In [2]:
# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"
#category_url = url_content.format(1)

他のカテゴリから取得する

In [3]:
category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

In [4]:
categories_url = []

for key, value in category.items():
    url = url_content.format(value)
    categories_url.append(url)

In [5]:
print(categories_url)

['https://schoo.jp/class/category/business/', 'https://schoo.jp/class/category/thinking/', 'https://schoo.jp/class/category/management/', 'https://schoo.jp/class/category/marketing/', 'https://schoo.jp/class/category/web_design/', 'https://schoo.jp/class/category/graphic/']


複数ページからも取ってこれるようにする

In [6]:
#categories_urls =[]

#for categories_url_pre in categories_url:
#    print(categories_url_pre+"?page=2")


for文を回す

In [7]:
#d_listに空リストを作成する
d_list = []

# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"

category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

categories_url = []

for key, value in category.items():
    url = url_content.format(value)
    categories_url.append(url)

    for category_url in categories_url:
        # レスポンスを取得する
        res = requests.get(category_url)

        # 取得結果をsoupに格納する
        soup = BeautifulSoup(res.text, 'html.parser', from_encoding='utf-8')

        #カテゴリとして講義内容を取ってくる
        contents = soup.find_all('div', class_='scrollContent listItem')

        # for文で各情報を取得する
        for content in contents:
            #講義内容の詳細へとべるリンク
            a_tag = content.find('a')
            url = a_tag.get('href')
            #使うときは以下の形でリンクに飛べる
            link = "https://schoo.jp"+url
            #講義大分類名
            big_title = content.find("h3", class_="title").text
            #関連ワード
            # 関連ワードを格納するリスト
            related_words = []

            # .find_all()で取得した結果をループ
            for words in content.find_all("a", class_="tag ga_link"):
                # 各要素からテキストを取得し、リストに追加
                related_words.append(words.text)
            # リスト内の各文字列から "n/" と空白を削除する
            related_words = [word.strip('\n ') for word in related_words]
            
            # linkにアクセスしてHTMLを取得
            detail = requests.get(link)
            # ステータスコードが200（成功）でない場合はエラーを出力して終了
            if detail.status_code != 200:
                print(f"Error: Failed to fetch the page. Status code: {detail.status_code}")
                exit()
            # 取得したHTMLをBeautifulSoupで解析
            soup_detail = BeautifulSoup(detail.text, 'html.parser')

            #コース概要
            #agenda_element = soup_detail.find("strong")
            # 要素が存在する場合は text 属性を取得、存在しない場合はデフォルトの値を使用
            #agenda = agenda_element.text if agenda_element else "Not available"


            #講師名
            teacher_name = soup_detail.find("h4", class_="p-courseTeachers__columnName").text
            #現職
            teacher_post = soup_detail.find("p", class_="p-courseTeachers__columnPost").text
            #経歴
            teacher_profile = soup_detail.find("p", class_="p-courseTeachers__columnProfile").text

            #講義詳細
            #講義リンク
            lecture_link_pre = soup_detail.find_all("li", class_="p-classes__item js-foldingByItemCountItem")
            # lecture_link_pre内の各要素からaタグを見つけてそのhref属性を取得
            lecture_links = [item.find('a').get('href') for item in lecture_link_pre]

            # 各要素に"https://schoo.jp"を前に追加して新しいリストに格納
            lecture_url = ["https://schoo.jp" + link for link in lecture_links]


            #講義タイトル
            middle_title_pre = soup_detail.find_all("h3", class_="p-classes__title")
            # middle_title_preから各要素のテキストを取得
            middle_titles = [middle_title_pre_1.text.strip() for middle_title_pre_1 in middle_title_pre]


            # 講義時間
            lecture_time_pre = soup_detail.find_all("span", class_="p-classes__onAirDuration")
            lecture_times = [lecture_time.text.strip() for lecture_time in lecture_time_pre]

            dic = {
                "link": link,
                "big_title": big_title,
                #"agenda": agenda,
                "lecture_link": lecture_url,
                "middle_titles": middle_titles,
                "lecture_time": lecture_times,
                "teacher_name": teacher_name,
                "teacher_post": teacher_post,
                "teacher_profile": teacher_profile,
                "related_words": related_words,
            }

            d_list.append(dic)

/Users/yakushijimana/anaconda3/envs/reskill/lib/python3.11/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [8]:
len(d_list)

882

In [9]:
max = range(0, len(contents))
df =pd.DataFrame(d_list)

In [10]:
df

,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,https://schoo.jp/course/7890,全1回「正しく伝わる」上司の会話術,[https://schoo.jp/class/10796],[相手の立場を尊重するアサーティブスキル],[60min],Tom RX,アサーティブ21代表,中学、高校時代を児童養護施設で暮らす。経済的困難の中で大学を卒業し、某大手航空会社に就職。航...,"[コミュニケーション術, リーダーシップ, チームビルディング]"
1,https://schoo.jp/course/7888,全1回緊張しても話せる３つの習慣,[https://schoo.jp/class/10787],[どうして緊張ってするんだろう？],[60min],丸山 久美子,株式会社シャベリーズ 代表取締役,もともと極度の緊張しいな性格。\n「人前で話せるようになりたい」と夢に抱き、2000年から展...,"[コミュニケーション術, プレゼンテーション, モチベーション]"
2,https://schoo.jp/course/7887,全1回禅でひも解く「怒り」のしずめかた,[https://schoo.jp/class/10786],[怒った自分を解放する],[60min],河口 智賢,禅僧／耕雲院副住職,曹洞宗耕雲院副住職／曹洞宗観音寺住職／NPO法人ぐんないや-織syoku-理事長／株式会社つ...,"[コミュニケーション術, 行動習慣]"
3,https://schoo.jp/course/7876,全1回結果を出す人のチームワーク術,[https://schoo.jp/class/10764],[数字を追うのに疲れたあなたへ〜仕事のやりがいを取り戻すための処方箋〜],[60min],加藤 芳久,経営コンサルタント,●「理念型育成®」を日本で初めて開発・体系化した人財育成の専門家。\n株式会社ファイブベイ、...,"[コミュニケーション術, ファシリテーション, マネジメント, リーダーシップ, 人材育成,..."
4,https://schoo.jp/course/7892,全1回頭がいい人に思わせるにはどうすればいいの,[https://schoo.jp/class/10800],[頭がいい人と思わせる質問の仕方],[30min],安田 正,（株）パンネーションズ・コンサルティング・グループ代表取締役,株式会社パンネーションズ・コンサルティング・グループ代表取締役\n一般社団法人コミュニケーシ...,"[コミュニケーション術, 行動習慣]"
...,...,...,...,...,...,...,...,...,...
877,https://schoo.jp/course/7281,全1回心を掴むデザインの分析,[https://schoo.jp/class/9379],[街中のデザインを教科書にする方法],[60min],前田 高志,株式会社NASU 代表取締役／前田デザイン室主宰／デザイナー,大阪芸術大学デザイン学科卒業後、任天堂株式会社へ入社。約15年プロモーションに携わったのち、...,"[クリエイティブデザイン, グラフィックデザイン, デザイン, プロダクトデザイン]"
878,https://schoo.jp/course/7300,全4回デザイン温故知新〜過去から現在、未来へ繋がるヒントを探る,"[https://schoo.jp/class/9425, https://schoo.jp...","[「これまでのデザイン」を理解するための基礎知識, 無印良品など「シンプルなデザイン」の原点...","[60min, 60min, 60min, 60min]",中村 将大,帝京平成大学 助教,帝京平成大学 助教。1983年福岡生まれ。2009年から2021年3月まで東洋美術学校専任講...,"[デザイン, 歴史, リベラルアーツ, デザイン力]"
879,https://schoo.jp/course/7344,全5回Figma入門,"[https://schoo.jp/class/9508, https://schoo.jp...","[Figma入門１～インストールして使ってみよう／カラースタイル、テキストスタイル～, Fi...","[30min, 45min, 45min, 30min, 25min]",山本 麻美,株式会社ロクナナ UI/UXデザイナー,インディーズ音楽プラットフォームを作りたい一心でWebを独学。企業からWeb制作の仕事を請け...,"[カンプ制作, スマホサイト制作, プロトタイピング, Figma]"
880,https://schoo.jp/course/7265,全1回超実践！ブランディングの現場,[https://schoo.jp/class/9350],[超実践！ブランディングの現場],[60min],西澤 明洋,ブランディングデザイナー,1976年滋賀県生まれ。株式会社エイトブランディングデザイン代表。 「ブランディングデザイン...,"[クリエイティブデザイン, ブランディング, デザイン経営]"


csvファイルに保存

In [11]:
# csvファイルに出力する
df.to_csv("schoo_v2.csv", index = False)

In [12]:
df.head(5)

,link,big_title,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,https://schoo.jp/course/7890,全1回「正しく伝わる」上司の会話術,[https://schoo.jp/class/10796],[相手の立場を尊重するアサーティブスキル],[60min],Tom RX,アサーティブ21代表,中学、高校時代を児童養護施設で暮らす。経済的困難の中で大学を卒業し、某大手航空会社に就職。航...,"[コミュニケーション術, リーダーシップ, チームビルディング]"
1,https://schoo.jp/course/7888,全1回緊張しても話せる３つの習慣,[https://schoo.jp/class/10787],[どうして緊張ってするんだろう？],[60min],丸山 久美子,株式会社シャベリーズ 代表取締役,もともと極度の緊張しいな性格。\n「人前で話せるようになりたい」と夢に抱き、2000年から展...,"[コミュニケーション術, プレゼンテーション, モチベーション]"
2,https://schoo.jp/course/7887,全1回禅でひも解く「怒り」のしずめかた,[https://schoo.jp/class/10786],[怒った自分を解放する],[60min],河口 智賢,禅僧／耕雲院副住職,曹洞宗耕雲院副住職／曹洞宗観音寺住職／NPO法人ぐんないや-織syoku-理事長／株式会社つ...,"[コミュニケーション術, 行動習慣]"
3,https://schoo.jp/course/7876,全1回結果を出す人のチームワーク術,[https://schoo.jp/class/10764],[数字を追うのに疲れたあなたへ〜仕事のやりがいを取り戻すための処方箋〜],[60min],加藤 芳久,経営コンサルタント,●「理念型育成®」を日本で初めて開発・体系化した人財育成の専門家。\n株式会社ファイブベイ、...,"[コミュニケーション術, ファシリテーション, マネジメント, リーダーシップ, 人材育成,..."
4,https://schoo.jp/course/7892,全1回頭がいい人に思わせるにはどうすればいいの,[https://schoo.jp/class/10800],[頭がいい人と思わせる質問の仕方],[30min],安田 正,（株）パンネーションズ・コンサルティング・グループ代表取締役,株式会社パンネーションズ・コンサルティング・グループ代表取締役\n一般社団法人コミュニケーシ...,"[コミュニケーション術, 行動習慣]"


取ってきたリンクごとにID（大分類ID）をつける

In [13]:
#d_listに空リストを作成する
d_list = []

# schooのURLを設定する
#カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"

category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

url_content = "https://schoo.jp/class/category/{}"

categories_url = {}

for key, value in category.items():
    url = url_content.format(value)
    categories_url[key] = url

print(categories_url)


{'ビジネススキル': 'https://schoo.jp/class/category/business/', '思考術・自己啓発': 'https://schoo.jp/class/category/thinking/', 'マネジメント': 'https://schoo.jp/class/category/management/', 'マーケティング': 'https://schoo.jp/class/category/marketing/', 'UI・UXデザイン': 'https://schoo.jp/class/category/web_design/', 'デザイン': 'https://schoo.jp/class/category/graphic/'}


In [11]:
# d_listに空リストを作成する
d_list = []

# schooのURLを設定する
# カテゴリごとになっているURL
url_content = "https://schoo.jp/class/category/{}"

category = {
    "ビジネススキル": "business/",
    "思考術・自己啓発": "thinking/",
    "マネジメント": "management/",
    "マーケティング": "marketing/",
    "UI・UXデザイン": "web_design/",
    "デザイン": "graphic/"
}

# categories_urlを辞書型に変更
categories_url = {}

for key, value in category.items():
    url = url_content.format(value)
    categories_url[key] = url

    for category_name, category_url in categories_url.items():
        # レスポンスを取得する
        res = requests.get(category_url)

        # 取得結果をsoupに格納する
        soup = BeautifulSoup(res.text, 'html.parser', from_encoding='utf-8')

        # カテゴリとして講義内容を取ってくる
        contents = soup.find_all('div', class_='scrollContent listItem')

        # for文で各情報を取得する
        for content in contents:
            # 講義内容の詳細へとべるリンク
            a_tag = content.find('a')
            url = a_tag.get('href')
            # 使うときは以下の形でリンクに飛べる
            link = "https://schoo.jp" + url
            # 講義大分類名
            big_title = content.find("h3", class_="title").text
            # イメージ画像
            img_tag = content.find('img')
            data_src_value = img_tag.get('data-src')
            # 関連ワード
            # 関連ワードを格納するリスト
            related_words = []

            # .find_all()で取得した結果をループ
            for words in content.find_all("a", class_="tag ga_link"):
                # 各要素からテキストを取得し、リストに追加
                related_words.append(words.text)
            # リスト内の各文字列から "n/" と空白を削除する
            related_words = [word.strip('\n ') for word in related_words]

            # linkにアクセスしてHTMLを取得
            detail = requests.get(link)
            # ステータスコードが200（成功）でない場合はエラーを出力して終了
            if detail.status_code != 200:
                print(f"Error: Failed to fetch the page. Status code: {detail.status_code}")
                exit()
            # 取得したHTMLをBeautifulSoupで解析
            soup_detail = BeautifulSoup(detail.text, 'html.parser')

            # コース概要
            # agenda_element = soup_detail.find("strong")
            # 要素が存在する場合は text 属性を取得、存在しない場合はデフォルトの値を使用
            # agenda = agenda_element.text if agenda_element else "Not available"

            # 講師名
            teacher_name = soup_detail.find("h4", class_="p-courseTeachers__columnName").text
            # 現職
            teacher_post = soup_detail.find("p", class_="p-courseTeachers__columnPost").text
            # 経歴
            teacher_profile = soup_detail.find("p", class_="p-courseTeachers__columnProfile").text

            # 講義詳細
            # 講義リンク
            lecture_link_pre = soup_detail.find_all("li", class_="p-classes__item js-foldingByItemCountItem")
            # lecture_link_pre内の各要素からaタグを見つけてそのhref属性を取得
            lecture_links = [item.find('a').get('href') for item in lecture_link_pre]

            # 各要素に"https://schoo.jp"を前に追加して新しいリストに格納
            lecture_url = ["https://schoo.jp" + link for link in lecture_links]

            # 講義タイトル
            middle_title_pre = soup_detail.find_all("h3", class_="p-classes__title")
            # middle_title_preから各要素のテキストを取得
            middle_titles = [middle_title_pre_1.text.strip() for middle_title_pre_1 in middle_title_pre]

            # 講義時間
            lecture_time_pre = soup_detail.find_all("span", class_="p-classes__onAirDuration")
            lecture_times = [lecture_time.text.strip() for lecture_time in lecture_time_pre]

            # カテゴリ名を追加した辞書
            dic = {
                "category": category_name,
                "link": link,
                "big_title": big_title,
                "image": data_src_value,
                # "agenda": agenda,
                "lecture_link": lecture_url,
                "middle_titles": middle_titles,
                "lecture_time": lecture_times,
                "teacher_name": teacher_name,
                "teacher_post": teacher_post,
                "teacher_profile": teacher_profile,
                "related_words": related_words,
            }

            d_list.append(dic)


/Users/yakushijimana/anaconda3/envs/reskill/lib/python3.11/site-packages/bs4/__init__.py:228: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [12]:
len(d_list)

882

In [13]:
max = range(0, len(contents))
df =pd.DataFrame(d_list)

In [14]:
df

,category,link,big_title,image,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,ビジネススキル,https://schoo.jp/course/7921,全1回誰もがやる気の種を持っている,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10837],[「心の本音」がやる気を育てる],[60min],藤由 達藏,夢実現応援家®,株式会社Gonmatus 代表取締役。夢実現応援家®。\nモットーは「人には無限の可能性があ...,"[モチベーション, 仕事術, 行動習慣]"
1,ビジネススキル,https://schoo.jp/course/7936,全1回今年やりたいことを達成する目標設定,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10859],[目標達成のための計画の立て方],[30min],吉武 麻子,タイムコーディネーター,時間×行動化のプロ\n著書：『目標や夢が達成できる 1 年・1 カ月・１週間・1 日の時間術...,"[時間術, ビジネス基礎力]"
2,ビジネススキル,https://schoo.jp/course/7934,全1回キャリアの荒波を乗り切るための対応力,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10856],[対応力を身につける3つのポイント],[30min],和田 裕美,作家・株式会社HIROWA 代表取締役,外資系教育会社での勤務経験から『世界No.2営業ウーマンの「売れる営業」に変わる本』を上梓し...,"[仕事術, ビジネス基礎力]"
3,ビジネススキル,https://schoo.jp/course/7911,全3回プロジェクトマネジメント - 周囲を上手に巻き込む仕事術,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,"[https://schoo.jp/class/10827, https://schoo.j...","[身近な仕事に学ぶ ゴール設定術, プロジェクト計画を立てよう, 「プロジェクト失敗あるある...",[60min],橋本 将功,パラダイスウェア株式会社 代表取締役,早稲田大学第一文学部卒業。文学修士(MA)。IT 業界24年目、PM歴23年目、経営歴13年...,"[マネジメント, ビジネスフレームワーク, 業務効率化]"
4,ビジネススキル,https://schoo.jp/course/7890,全1回「正しく伝わる」上司の会話術,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10796],[相手の立場を尊重するアサーティブスキル],[60min],Tom RX,アサーティブ21代表,中学、高校時代を児童養護施設で暮らす。経済的困難の中で大学を卒業し、某大手航空会社に就職。航...,"[コミュニケーション術, リーダーシップ, チームビルディング]"
...,...,...,...,...,...,...,...,...,...,...,...
877,デザイン,https://schoo.jp/course/7281,全1回心を掴むデザインの分析,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/9379],[街中のデザインを教科書にする方法],[60min],前田 高志,株式会社NASU 代表取締役／前田デザイン室主宰／デザイナー,大阪芸術大学デザイン学科卒業後、任天堂株式会社へ入社。約15年プロモーションに携わったのち、...,"[クリエイティブデザイン, グラフィックデザイン, デザイン, プロダクトデザイン]"
878,デザイン,https://schoo.jp/course/7300,全4回デザイン温故知新〜過去から現在、未来へ繋がるヒントを探る,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,"[https://schoo.jp/class/9425, https://schoo.jp...","[「これまでのデザイン」を理解するための基礎知識, 無印良品など「シンプルなデザイン」の原点...","[60min, 60min, 60min, 60min]",中村 将大,帝京平成大学 助教,帝京平成大学 助教。1983年福岡生まれ。2009年から2021年3月まで東洋美術学校専任講...,"[デザイン, 歴史, リベラルアーツ, デザイン力]"
879,デザイン,https://schoo.jp/course/7344,全5回Figma入門,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,"[https://schoo.jp/class/9508, https://schoo.jp...","[Figma入門１～インストールして使ってみよう／カラースタイル、テキストスタイル～, Fi...","[30min, 45min, 45min, 30min, 25min]",山本 麻美,株式会社ロクナナ UI/UXデザイナー,インディーズ音楽プラットフォームを作りたい一心でWebを独学。企業からWeb制作の仕事を請け...,"[カンプ制作, スマホサイト制作, プロトタイピング, Figma]"
880,デザイン,https://schoo.jp/course/7265,全1回超実践！ブランディングの現場,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/9350],[超実践！ブランディングの現場],[60min],西澤 明洋,ブランディングデザイナー,1976年滋賀県生まれ。株式会社エイトブランディングデザイン代表。 「ブランディングデザイン...,"[クリエイティブデザイン, ブランディング, デザイン経営]"


In [7]:
df.head(5)

,category,link,big_title,image,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words
0,ビジネススキル,https://schoo.jp/course/7921,全1回誰もがやる気の種を持っている,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10837],[「心の本音」がやる気を育てる],[60min],藤由 達藏,夢実現応援家®,株式会社Gonmatus 代表取締役。夢実現応援家®。\nモットーは「人には無限の可能性があ...,"[モチベーション, 仕事術, 行動習慣]"
1,ビジネススキル,https://schoo.jp/course/7936,全1回今年やりたいことを達成する目標設定,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10859],[目標達成のための計画の立て方],[30min],吉武 麻子,タイムコーディネーター,時間×行動化のプロ\n著書：『目標や夢が達成できる 1 年・1 カ月・１週間・1 日の時間術...,"[時間術, ビジネス基礎力]"
2,ビジネススキル,https://schoo.jp/course/7934,全1回キャリアの荒波を乗り切るための対応力,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10856],[対応力を身につける3つのポイント],[30min],和田 裕美,作家・株式会社HIROWA 代表取締役,外資系教育会社での勤務経験から『世界No.2営業ウーマンの「売れる営業」に変わる本』を上梓し...,"[仕事術, ビジネス基礎力]"
3,ビジネススキル,https://schoo.jp/course/7911,全3回プロジェクトマネジメント - 周囲を上手に巻き込む仕事術,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,"[https://schoo.jp/class/10827, https://schoo.j...","[身近な仕事に学ぶ ゴール設定術, プロジェクト計画を立てよう, 「プロジェクト失敗あるある...",[60min],橋本 将功,パラダイスウェア株式会社 代表取締役,早稲田大学第一文学部卒業。文学修士(MA)。IT 業界24年目、PM歴23年目、経営歴13年...,"[マネジメント, ビジネスフレームワーク, 業務効率化]"
4,ビジネススキル,https://schoo.jp/course/7890,全1回「正しく伝わる」上司の会話術,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10796],[相手の立場を尊重するアサーティブスキル],[60min],Tom RX,アサーティブ21代表,中学、高校時代を児童養護施設で暮らす。経済的困難の中で大学を卒業し、某大手航空会社に就職。航...,"[コミュニケーション術, リーダーシップ, チームビルディング]"


In [15]:
import numpy as np

# カテゴリに応じてIDを割り当てるための条件
conditions = [
    (df['category'] == "ビジネススキル"),
    (df['category'] == "思考術・自己啓発"),
    (df['category'] == "マネジメント"),
    (df['category'] == "マーケティング"),
    (df['category'] == "UI・UXデザイン"),
    (df['category'] == "デザイン")
]

# 各条件に対応するID
values = [1, 2, 3, 4, 5, 6]

# 'category_id'列に条件に応じてIDを割り当てる
df['category_id'] = np.select(conditions, values)

# 結果を表示
df


,category,link,big_title,image,lecture_link,middle_titles,lecture_time,teacher_name,teacher_post,teacher_profile,related_words,category_id
0,ビジネススキル,https://schoo.jp/course/7921,全1回誰もがやる気の種を持っている,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10837],[「心の本音」がやる気を育てる],[60min],藤由 達藏,夢実現応援家®,株式会社Gonmatus 代表取締役。夢実現応援家®。\nモットーは「人には無限の可能性があ...,"[モチベーション, 仕事術, 行動習慣]",1
1,ビジネススキル,https://schoo.jp/course/7936,全1回今年やりたいことを達成する目標設定,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10859],[目標達成のための計画の立て方],[30min],吉武 麻子,タイムコーディネーター,時間×行動化のプロ\n著書：『目標や夢が達成できる 1 年・1 カ月・１週間・1 日の時間術...,"[時間術, ビジネス基礎力]",1
2,ビジネススキル,https://schoo.jp/course/7934,全1回キャリアの荒波を乗り切るための対応力,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10856],[対応力を身につける3つのポイント],[30min],和田 裕美,作家・株式会社HIROWA 代表取締役,外資系教育会社での勤務経験から『世界No.2営業ウーマンの「売れる営業」に変わる本』を上梓し...,"[仕事術, ビジネス基礎力]",1
3,ビジネススキル,https://schoo.jp/course/7911,全3回プロジェクトマネジメント - 周囲を上手に巻き込む仕事術,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,"[https://schoo.jp/class/10827, https://schoo.j...","[身近な仕事に学ぶ ゴール設定術, プロジェクト計画を立てよう, 「プロジェクト失敗あるある...",[60min],橋本 将功,パラダイスウェア株式会社 代表取締役,早稲田大学第一文学部卒業。文学修士(MA)。IT 業界24年目、PM歴23年目、経営歴13年...,"[マネジメント, ビジネスフレームワーク, 業務効率化]",1
4,ビジネススキル,https://schoo.jp/course/7890,全1回「正しく伝わる」上司の会話術,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/10796],[相手の立場を尊重するアサーティブスキル],[60min],Tom RX,アサーティブ21代表,中学、高校時代を児童養護施設で暮らす。経済的困難の中で大学を卒業し、某大手航空会社に就職。航...,"[コミュニケーション術, リーダーシップ, チームビルディング]",1
...,...,...,...,...,...,...,...,...,...,...,...,...
877,デザイン,https://schoo.jp/course/7281,全1回心を掴むデザインの分析,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/9379],[街中のデザインを教科書にする方法],[60min],前田 高志,株式会社NASU 代表取締役／前田デザイン室主宰／デザイナー,大阪芸術大学デザイン学科卒業後、任天堂株式会社へ入社。約15年プロモーションに携わったのち、...,"[クリエイティブデザイン, グラフィックデザイン, デザイン, プロダクトデザイン]",6
878,デザイン,https://schoo.jp/course/7300,全4回デザイン温故知新〜過去から現在、未来へ繋がるヒントを探る,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,"[https://schoo.jp/class/9425, https://schoo.jp...","[「これまでのデザイン」を理解するための基礎知識, 無印良品など「シンプルなデザイン」の原点...","[60min, 60min, 60min, 60min]",中村 将大,帝京平成大学 助教,帝京平成大学 助教。1983年福岡生まれ。2009年から2021年3月まで東洋美術学校専任講...,"[デザイン, 歴史, リベラルアーツ, デザイン力]",6
879,デザイン,https://schoo.jp/course/7344,全5回Figma入門,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,"[https://schoo.jp/class/9508, https://schoo.jp...","[Figma入門１～インストールして使ってみよう／カラースタイル、テキストスタイル～, Fi...","[30min, 45min, 45min, 30min, 25min]",山本 麻美,株式会社ロクナナ UI/UXデザイナー,インディーズ音楽プラットフォームを作りたい一心でWebを独学。企業からWeb制作の仕事を請け...,"[カンプ制作, スマホサイト制作, プロトタイピング, Figma]",6
880,デザイン,https://schoo.jp/course/7265,全1回超実践！ブランディングの現場,//s3-ap-northeast-1.amazonaws.com/i.schoo/imag...,[https://schoo.jp/class/9350],[超実践！ブランディングの現場],[60min],西澤 明洋,ブランディングデザイナー,1976年滋賀県生まれ。株式会社エイトブランディングデザイン代表。 「ブランディングデザイン...,"[クリエイティブデザイン, ブランディング, デザイン経営]",6


In [16]:
# csvファイルに出力する
df.to_csv("schoo_v5.csv", index = False)